In [ ]:
#from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "accuser"
password = "password2"
shelter = AnimalShelter()


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))





#########################
# Dashboard Layout / View
#########################
app = dash.Dash('SimpleExample')


#display the logo for the entire project
image_filename = 'GraziosoSalvareLogo.png' 

encoded_image = base64.b64encode(open(image_filename, 'rb').read())


#layed out table to display the animal data, like name breed etc
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),#name of the entire dashboard in header 1 style
    html.Center(html.B(html.H3('Made by Ismael DeRocco'))), #my name, the programmer
    html.Hr(),
    #buttons to select which type of animal is going to be viewed
    html.Div(className = 'row',style={'display' : 'flex'},
            children=[
                html.Button(id='button1',n_clicks=0, children= 'Water Rescue'),
                html.Button(id='button2',n_clicks=0, children= 'Mountain or Wilderness Rescue'),
                html.Button(id='button3',n_clicks=0, children='Disaster Rescue or Individual Tracking'),
                html.Button(id='button4', n_clicks=0, children='reset')
            ])
    dash_table.DataTable(
        id='datatable-id',
        data=df.to_dict('records'),
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        #Added a set amount of names that can be set into each list for a user-friendly experience.
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        page_size = 10

    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',style={'display' : 'flex'},
             children=[
                 html.Div(
                     id='graph-id',
                     className='col s12 m6',),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',)
             ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback(Output('datatable-id',"data"),
             [Input('button1', 'n_clicks'),
              Input('button2', 'n_clicks'),
              Input('button3', 'n_clicks'), 
              Input('button4', 'n_clicks')
             ])
def on_click(button1, button2, button3, button4):
    if(int(button1) >= 1):
        df = pd.Dataframe.from_records(shelter.read({'$and': [
            {'$or': [ 
                {'breed':'Labrador Retriever Mix'},
                {'breed':'Chesapeake Bay Retriever'},
                {'breed':'Newfoundland'}]},
            {'sex_upon_outcome':'Intact Female'},
            {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
        button2, button3, button4 = 0 #resets counter on all buttons to prevent collision
    elif (int(button2) >=1):
        df = pd.Dataframe.from_records(shelter.read({'$and': [
            {'$or': [ {'breed':'German Shepherd'},
                     {'breed':'Alaskan Malamute'},
                     {'breed':'Old English Sheepdog'},
                     {'breed':'Siberian Husky'},
                     {'breed':'Rottweiler'}]},
            {'sex_upon_outcome':'Intact Male'},
            {'age_upon_outcome_in_weeks':{'$lte':26, 'gte':156}}]}))
        button1, button3, button4 = 0
    elif (int(button3) >=1 ):
        df = pd.Dataframe.from_records(shelter.read({'$and': [
            {'$or': [ {'breed':'Doberman Pinscher'},
                      {'breed':'German Sheperd'},
                      {'breed':'Golden Retriever'},
                      {'breed':'Bloodhound'}, 
                      {'breed':'Rottweiler'}]},
            {'sex_upon_outcome':'Intact Male'},
            {'age_upon_outcome_in_weeks':{'$lte':20, 'gte':300}}]}))
        button1, button2, button 4 = 0
        
    elif (int(button4) >=1 ):
        df = pd.Dataframe.from_records(shelter.read())
        button1, button2, button3 = 0
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        return df.to_dir('records')
        
    #call back for the piechart graph
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])


def update_graph(viewData):
    #code for the piechart
    df = pd.DataFrame.from_dict(viewData)
    return [
        dcc.graph(
        figure = px.pie(df, values = values, names = names, title = 'Percentage of breeds avaliable'))
    ]



#call back for map widget
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")]
)

def update_map(viewData):
#FIXME Add in the code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    #added map for location of each animal. The pin will be dropped on the specified location
    newLat = dff.loc[1,'location_lat']
    newLon = dff.loc[1,'location_long']
    return [
        dl.Map(style={'width': '800px', 'height': '300px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[newLat,newLon], children=[
                dl.Tooltip(dff.loc[1,'breed']),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[1,'name'])
                    #pin will have "animal name" then when selected will state animals actual name
                ])
            ])
        ])
    ]

#regular Dash executable 
if __name__ == '__main__' :
    app.run_server()